# 01 - Fundamentos de Python para meteorología

Este notebook cubre variables, listas, diccionarios, condicionales y funciones.

In [ ]:
# Celda de instalación (Colab)
try:
    import google.colab
    !pip install -r https://raw.githubusercontent.com/jpantojac/wrf-course/main/requirements_colab.txt

except Exception:
    pass

## Tipos básicos y estructuras

In [ ]:
a = 10
b = 3.5
name = 'Estacion CATAM'
flag = True
print(a,b,name,flag)

## Listas y diccionarios
Representar estaciones y valores de temperatura.

In [ ]:
stations = ['CATAM','Fortaleza','Rionegro']
temps = [22.1,21.3,19.5]
obs = dict(zip(stations, temps))
for s,t in obs.items():
    print(f"{s}: {t} °C")

## Funciones y uso práctico

In [ ]:
def c_to_k(temp_c):
    return temp_c + 273.15

print('CATAM (K):', c_to_k(obs['CATAM']))